In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import numpy as np
import tensorflow as tf

class LogicGate(tf.Module):
    def __init__(self):
        super().__init__()
        self.built = False

    def __call__(self, x, train=True):
        if not self.built:
            input_dim = x.shape[-1]
            hidden_dim = 2


            self.w1 = tf.Variable(tf.random.normal([input_dim, hidden_dim]), name="hidden weights")
            self.b1 = tf.Variable(tf.zeros([hidden_dim]), name="hidden bias")

            self.w2 = tf.Variable(tf.random.normal([hidden_dim, 1]), name="output weights")
            self.b2 = tf.Variable(tf.zeros([1]), name="output bias")

            self.built = True

        hidden = tf.sigmoid(tf.add(tf.matmul(x, self.w1), self.b1))
        output = tf.sigmoid(tf.add(tf.matmul(hidden, self.w2), self.b2))
        return output

def compute_loss(y_pred, y_true):
    epsilon = 1e-7
    y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
    return -tf.reduce_mean(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))

def train_model(model, x_train, y_train, learning_rate=0.1, epochs=6000):
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            y_pred = model(x_train)
            loss = compute_loss(y_pred, y_train)

        grads = tape.gradient(loss, model.variables)
        for g, v in zip(grads, model.variables):
            v.assign_sub(learning_rate * g)

        if epoch % 1000 == 0:
            acc = compute_accuracy(model, x_train, y_train)
            print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}, Accuracy: {acc:.4f}")

def compute_accuracy(model, x, y_true):
    y_pred = model(x, train=False)
    y_pred_rounded = tf.round(y_pred)
    correct = tf.equal(y_pred_rounded, y_true)
    return tf.reduce_mean(tf.cast(correct, tf.float32)).numpy()


In [3]:

xor_table = np.array([[0, 0, 0],
                      [0, 1, 1],
                      [1, 0, 1],
                      [1, 1, 0]], dtype=np.float32)

x_train = xor_table[:, :2]
y_train = xor_table[:, 2:]

model = LogicGate()
train_model(model, x_train, y_train)

w1 = model.w1.numpy()
b1 = model.b1.numpy()
w2 = model.w2.numpy()
b2 = model.b2.numpy()

print(f"Hidden Layer Weights (w1): \n{w1}")
print(f"Hidden Layer Bias (b1): \n{b1}")
print(f"Output Layer Weights (w2): \n{w2}")
print(f"Output Layer Bias (b2): \n{b2}\n")

y_pred = model(x_train, train=False).numpy().round().astype(np.uint8)
print("Predicted Truth Table:")
print(np.column_stack((xor_table[:, :2], y_pred)))


Epoch 0, Loss: 0.7005, Accuracy: 0.5000
Epoch 1000, Loss: 0.6478, Accuracy: 0.5000
Epoch 2000, Loss: 0.4644, Accuracy: 0.7500
Epoch 3000, Loss: 0.1412, Accuracy: 1.0000
Epoch 4000, Loss: 0.0646, Accuracy: 1.0000
Epoch 5000, Loss: 0.0404, Accuracy: 1.0000
Hidden Layer Weights (w1): 
[[ 5.867798 -5.50227 ]
 [-6.042031  5.181547]]
Hidden Layer Bias (b1): 
[-3.307535 -2.862797]
Output Layer Weights (w2): 
[[8.383733]
 [8.439888]]
Output Layer Bias (b2): 
[-4.1069164]

Predicted Truth Table:
[[0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
